In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import pandas as pd
from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, map_keys, col, first, get_json_object, array, to_json, struct

In [3]:
# Spark 세션 생성
spark = SparkSession \
        .builder \
        .appName("processingJson") \
        .getOrCreate()

23/11/17 16:35:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
# 분할된 JSON 파일 경로 선택하는 함수
def nth_json_path(n):
    return f'/Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta/naverplace_meta_{n}.json'

In [5]:
# 첫번째 JSON 파일 경로
n = 1
json_path_1 = nth_json_path(n)
json_path_1

'/Users/b06/Desktop/yeardream/medi-05/data/naverplace_meta/naverplace_meta_1.json'

In [6]:
data = spark.read.json(json_path_1)

23/11/17 16:35:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
## EDA
#df.show(1)
#df.printSchema(1)
#df.dtypes
columns = data.columns

In [8]:
hb_columns = [c for c in columns if "HospitalBase" in c]
hb_columns

['HospitalBase:11779766',
 'HospitalBase:1233961055',
 'HospitalBase:12857046',
 'HospitalBase:1344046290',
 'HospitalBase:1413208872',
 'HospitalBase:1934828030',
 'HospitalBase:19518318',
 'HospitalBase:19530874',
 'HospitalBase:265615453',
 'HospitalBase:33237939']

In [9]:
hb_column = hb_columns.pop(0)
hb_column

'HospitalBase:11779766'

In [10]:
df = data.select(hb_column)
df.printSchema(2)

root
 |-- HospitalBase:11779766: struct (nullable = true)
 |    |-- __typename: string (nullable = true)
 |    |-- address: string (nullable = true)
 |    |-- bookingBusinessId: string (nullable = true)
 |    |-- bookingButtonName: string (nullable = true)
 |    |-- bookingDisplayName: string (nullable = true)
 |    |-- bookingHubButtonName: string (nullable = true)
 |    |-- broadcastInfos: string (nullable = true)
 |    |-- businessHours: array (nullable = true)
 |    |-- category: string (nullable = true)
 |    |-- categoryCode: string (nullable = true)
 |    |-- categoryCodeList: array (nullable = true)
 |    |-- categoryCount: long (nullable = true)
 |    |-- cescoCheck: string (nullable = true)
 |    |-- cescoLink: string (nullable = true)
 |    |-- cescofsCheck: string (nullable = true)
 |    |-- cescofsLink: string (nullable = true)
 |    |-- chatBotUrl: string (nullable = true)
 |    |-- conveniences: array (nullable = true)
 |    |-- coordinate: struct (nullable = true)
 |   

In [11]:
targets = [
    'name', 
    'road', 
    'reviewSettings', 
    'conveniences', 
    'keywords', 
    'phone', 
    'virtualPhone', 
    'naverBookingUrl', 
    'talktalkUrl', 
    'paymentInfo', 
    'homepages',
    'visitorReviewsTotal',
    'description',
    'Images'
]
targets

['name',
 'road',
 'reviewSettings',
 'conveniences',
 'keywords',
 'phone',
 'virtualPhone',
 'naverBookingUrl',
 'talktalkUrl',
 'paymentInfo',
 'homepages',
 'visitorReviewsTotal',
 'description',
 'Images']

In [12]:
sel_cols = [col(hb_column + "." + t).alias(t) for t in targets]
sel_cols

[Column<'HospitalBase:11779766.name AS name'>,
 Column<'HospitalBase:11779766.road AS road'>,
 Column<'HospitalBase:11779766.reviewSettings AS reviewSettings'>,
 Column<'HospitalBase:11779766.conveniences AS conveniences'>,
 Column<'HospitalBase:11779766.keywords AS keywords'>,
 Column<'HospitalBase:11779766.phone AS phone'>,
 Column<'HospitalBase:11779766.virtualPhone AS virtualPhone'>,
 Column<'HospitalBase:11779766.naverBookingUrl AS naverBookingUrl'>,
 Column<'HospitalBase:11779766.talktalkUrl AS talktalkUrl'>,
 Column<'HospitalBase:11779766.paymentInfo AS paymentInfo'>,
 Column<'HospitalBase:11779766.homepages AS homepages'>,
 Column<'HospitalBase:11779766.visitorReviewsTotal AS visitorReviewsTotal'>,
 Column<'HospitalBase:11779766.description AS description'>,
 Column<'HospitalBase:11779766.Images AS Images'>]

In [13]:
sel_df = df.select(sel_cols)
sel_df.printSchema(1)

root
 |-- name: string (nullable = true)
 |-- road: string (nullable = true)
 |-- reviewSettings: struct (nullable = true)
 |-- conveniences: array (nullable = true)
 |-- keywords: array (nullable = true)
 |-- phone: string (nullable = true)
 |-- virtualPhone: string (nullable = true)
 |-- naverBookingUrl: string (nullable = true)
 |-- talktalkUrl: string (nullable = true)
 |-- paymentInfo: array (nullable = true)
 |-- homepages: struct (nullable = true)
 |-- visitorReviewsTotal: long (nullable = true)
 |-- description: string (nullable = true)
 |-- Images: array (nullable = true)



In [14]:
string_cols = [
    'name', 
    'road',
    'phone',
    'virtualPhone',
    'naverBookingUrl',
    'talktalkUrl',
    'description'
]

In [15]:
struct_cols = [
    'reviewSettings',
    'homepages'
]

In [16]:
array_cols = [
    'conveniences',
    'keywords',
    'Images'
]

In [17]:
sel_df = sel_df.select(string_cols).filter(~col('name').isNull())
pd_df = sel_df.toPandas()
pd_df

,name,road,phone,virtualPhone,naverBookingUrl,talktalkUrl,description
0,다인한의원,"지하철 숙대입구역 10번에서 뒤로 돌아 숙대 방향으로 500M 올라오시거나, 400...",02-711-9557,,https://m.booking.naver.com/booking/13/bizes/4...,None,"숙대 앞 청파로에 위치한 한방재활의학과 전문의, 한의학 박사가 진료하는 한의원입니다..."
